In [ ]:
# Step-1. 필요한 모듈, 라이브러리 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image as XLImage
from selenium.webdriver.chrome.options import Options
from deep_translator import GoogleTranslator
import undetected_chromedriver as uc
import os
import time
import math
import pandas as pd
import re
import urllib.request
import urllib
import random
import tempfile



# Step-2. 사용자에게 조회 내용 / 저장할 폴더명 입력 받기
now = time.localtime()
t_time = '%04d-%02d-%02d-%02d-%02d-%02d' % (
    now.tm_year, now.tm_mon, now.tm_mday,
    now.tm_hour, now.tm_min, now.tm_sec
)


print('='*120)
print('Shopee 상품정보 수집하는 크롤러 프로그램')
print('='*120)


sch_num = int(input('1.크롤링할 상품 수를 입력하세요: '))
f_dir_name = input('2.파일을 저장할 폴더명만 쓰세요: ')
print('='*120)

if f_dir_name =='' :
    f_dir = "C:\\py_temp\\web_test\\Shopee\\"
else :
    f_dir = "C:\\py_temp\\web_test\\Shopee\\" + f_dir_name

os.makedirs(f_dir)

img_dir = f_dir+"\\images"
os.makedirs(img_dir)

fx_name=f_dir+ '\\' + 'Shopee' + '-' + t_time +'.xlsx'
fc_name=f_dir+ '\\' + 'Shopee' + '-' + t_time +'.csv'



# Step-3. 크롬 드라이버를 사용해서 웹브라우저 실행
# url='https://shopee.com/'

# s_time = time.time()

# s = Service("c:/py_temp/chromedriver.exe")
# driver = webdriver.Chrome(service=s)

# driver.get(url)
# driver.maximize_window()
# time.sleep(3)

# Step-3. 크롬 드라이버를 사용해서 웹브라우저 실행  undetected chromedriver 설정하기
CHROME_PATH = r"C:\Program Files\Google\Chrome\Application\chrome.exe"  # 크롬 경로 확인!
VERSION_MAIN = 142  # 현재 크롬 메이저 버전

options = Options()
# 필요하면 headless: 많은 사이트가 탐지하니 처음엔 꺼두고 테스트
# options.add_argument("--headless=new")

# 깨끗한 임시 프로필(기존 프로필 충돌 방지)
user_data = os.path.join(tempfile.gettempdir(), "uc_profile_clean")
os.makedirs(user_data, exist_ok=True)

# from selenium import webdriver
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver= webdriver.Chrome()

driver = uc.Chrome(
    options=options,
    browser_executable_path=CHROME_PATH,
    version_main=VERSION_MAIN,      # ← 핵심: 140으로 맞추기
    user_data_dir=user_data,
    use_subprocess=True,           # 안 되면 True로 바꿔 재시도
    patcher_force_close=True,       # 패쳐가 점유 중일 때 강제 종료
    debug=True,
)

driver.get("https://shopee.vn/Makeup-cat.11036279.11036314")
driver.maximize_window()
time.sleep(3)

def scroll_down(driver):
    driver.execute_script("window.scrollBy(0,1000);")
    time.sleep(4)


# # 로그인용 시간
# time.sleep(30)

Shopee 상품정보 수집하는 크롤러 프로그램


1.크롤링할 상품 수를 입력하세요:  100
2.파일을 저장할 폴더명만 쓰세요:  555


In [ ]:
# Step-5. 내용 추출
rank2 = []            # 판매 순위
title2 = []           # 상품 제목
price2 = []           # 원래 가격
star2 = []            # 별점
review2 = []          # 리뷰수
con2 = []             # 제품 설명
item_url2 = []        # 상품 url

count = 0
img_file_no = 0

for i in range(1, 6) :
    scroll_down(driver)

print('요청해주신 조건으로 Shopee 수집을 시작합니다:D')


# 상품제목, 가격, 링크
while count < sch_num :
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        slist = soup.find('ul','row shopee-search-item-result__items').find_all('li','col-xs-2-4 shopee-search-item-result__item')

        for i in range(len(slist)) :
            if count == sch_num :
                break

            li = slist[i]

            count += 1
            img_file_no += 1

            print('-'*120)
            time.sleep(2)

            # 번호
            rank2.append(count)



            try :
                title_div = li.find('div','line-clamp-2 break-words min-w-0 min-h-[2.5rem] text-sm th:text-[12px] my:text-[12px] km:text-[12px]')
            except :
                title_div = li.select_one("div[class*='line-clamp']")

            # 1) 제목 div 자체가 없는 카드 → 광고/빈칸일 가능성 → 그냥 스킵
            if title_div is None:
                print("제목 div 없음 → 이 상품은 건너뜀")
                continue

            # 2) div 안에는 쓸데없는 문자나 공백이 섞여 있을 수 있으므로
            #    텍스트 조각들을 리스트로 뽑아서 마지막 조각을 제목으로 사용
            texts = [t for t in title_div.stripped_strings]

            if not texts:
                print("제목 텍스트 없음 → 건너뜀")
                continue

            title = texts[-1]   # 보통 마지막 조각이 실제 상품명

            print("상품제목:", title)
            title2.append(title)

            # # 제목
            # title_tag = li.select_one("div[class*='line-clamp-2']")
            # title = title_tag.get_text().strip()
            # print('상품제목:',title)
            # title2.append(title)


            # 현재 가격
            price = li.find("span", class_="truncate text-base/5 font-medium").get_text().strip()
            price = price + "₫"
            print('상품가격:',price)
            price2.append(price)


            # 이미지
            time.sleep(1)
            photo_tag = li.find('div','w-full relative z-0').find('img')
            photo = photo_tag.get('src')
            filename = os.path.join(img_dir, str(img_file_no) + '.jpg')
            urllib.request.urlretrieve(photo, filename)
            # print('%s번째 이미지 저장완료' % img_file_no)
            time.sleep(1)


            # 상세페이지 url
            item_url_href = li.find('a')['href']
            item_url = 'https://shopee.vn'+item_url_href
            print(f"{count}번째 상품 url:{item_url}")
            item_url2.append(item_url)
            time.sleep(3)

            if count % 60 == 0:
                driver.find_element(By.CSS_SELECTOR,'#main > div > div.t5pFIU > div > div > div > div > div.container.cKV3cM > div.u0v0QW > section > fieldset > div.shopee-mini-page-controller > a.shopee-button-outline.shopee-mini-page-controller__next-btn > svg').click()
                time.sleep(5)

        if count == sch_num :
            break


요청해주신 조건으로 Shopee 수집을 시작합니다:D
------------------------------------------------------------------------------------------------------------------------
상품제목: Kem che khuyết điểm kiềm dầu bền màu Lemonade Matte Addict Concealer 3g
상품가격: 175.000₫
1번째 상품 url:https://shopee.vn/Kem-che-khuyết-điểm-kiềm-dầu-bền-màu-Lemonade-Matte-Addict-Concealer-3g-i.99553806.18560602714?extraParams=%7B%22display_model_id%22%3A197399422867%2C%22model_selection_logic%22%3A3%7D
------------------------------------------------------------------------------------------------------------------------
상품제목: Son Kem Lì Black Rouge Air Fit Velvet Tint
상품가격: 129.000₫
2번째 상품 url:https://shopee.vn/Son-Kem-Lì-Black-Rouge-Air-Fit-Velvet-Tint-i.8516029.1414083985?extraParams=%7B%22display_model_id%22%3A132093659997%2C%22model_selection_logic%22%3A3%7D
------------------------------------------------------------------------------------------------------------------------
상품제목: Phấn Phủ Mịn Lì Kiềm Dầu Fit Me Matte Poreless 

In [ ]:
# 상세페이지 - 별점, 리뷰수, 제품설명
con_num = 0

for i in item_url2 :
    driver.get(i)
    scroll_down(driver)
    time.sleep(3)

    con_num += 1

    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')
    time.sleep(5)

    if con_num % 20 == 0:
        time.sleep(random.uniform(8, 15))


    print('-'*120)
    print(f"{con_num}번째 상품 상세페이지 접속")

    # 별점
    star = soup2.find("div","F9RHbS dQEiAI jMXp4d").get_text().strip()
    print("별점:", star)
    star2.append(star)
    time.sleep(random.uniform(1, 3))


    # 리뷰수
    review_tag = soup2.find_all("div","F9RHbS")
    review = review_tag[1].get_text().strip()
    print("리뷰수:", review)
    review2.append(review)
    time.sleep(random.uniform(1, 3))

    # 제품 설명

    con_list = soup2.find("div",'product-detail page-product__detail').find_all("section")
    con = con_list[1].find("div","e8lZp3").get_text().strip()
    print("제품설명:", con)
    con2.append(con)
    time.sleep(random.uniform(1, 3))
print("/n상세페이지 크롤링 완료되었습니다:D")


------------------------------------------------------------------------------------------------------------------------
1번째 상품 상세페이지 접속
별점: 4.9
리뷰수: 23,4k
제품설명: THÔNG TIN THƯƠNG HIỆU:LEMONADE MANG ĐẾNGiải Pháp Trang Điểm Dễ Dàng Cho Phụ Nữ ViệtDựa trên kinh nghiệm 15 năm chinh chiến trong ngành làm đẹp và hợp tác với các tập đoàn mỹ phẩm nổi tiếng trên Thế giới, Makeup Artist Quách Ánh cùng những cộng sự của mình đã tạo nên thương hiệu mỹ phẩm Lemonade. Với các dòng sản phẩm đa công năng và tiện dụng được nghiên cứu dựa trên khí hậu và làn da của phụ nữ Việt, Lemonade giúp bạn hoàn thiện vẻ đẹp một cách nhanh chóng và dễ dàng hơn: Dễ dàng sử dụng, dễ dàng kết hợp và dễ dàng mang đi.
Thời gian sử dụng tốt nhất: 06 tháng từ khi mở nắpHSD & NSX: 03 năm kể từ ngày SX (ghi trên bao bì)Xuất xứ thương hiệu: Việt NamSản xuất tại: Đài LoanKhối lượng: 3 gramThành phần: Xem chi tiết trên bao bì sản phẩm
---------------------------------------------------------------------------------------------

AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
# Step-6. xlsx 형태로 저장

# con = GoogleTranslator(source='vi', target='en').translate(con)

Shopee = pd.DataFrame()
Shopee['순위']=pd.Series(rank2)
Shopee['상품이름']=pd.Series(title2)
Shopee['판매가격']=pd.Series(price2)
Shopee['별점']=pd.Series(star2)
Shopee['리뷰개수']=pd.Series(review2)
Shopee['상세내용']=pd.Series(con2)
Shopee['url']=pd.Series(item_url2)


Shopee.to_excel(fx_name ,index=False , engine='openpyxl')
Shopee.to_csv(fc_name, index=False, encoding="utf-8-sig")



# Step-7. 생성된 파일에 이미지 추가 / 데이터에 맞게 너비 조절
wb = load_workbook(fx_name)
ws = wb["Sheet1"]

ws.insert_cols(2)  # 2번째 열(B열)에 새로운 열 삽입
ws.cell(row=1, column=2, value="상품이미지")

ws.column_dimensions['B'].width = 25
row_cnt = sch_num + 1

# 행 높이 설정
for r in range(2, row_cnt + 1):
    ws.row_dimensions[r].height = 120

# 이미지 삽입
for i in range(1, sch_num + 1):
    img_path = os.path.join(img_dir, f"{i}.jpg")
    if not os.path.exists(img_path):
        continue

    img = XLImage(img_path)
    img.width = 100
    img.height = 100

    cell_addr = f"B{i+1}"  # B열, i+1행
    ws.add_image(img, cell_addr)



# 저장
wb.save(fx_name)



# Step-8. 요약 정보 출력
# print("=" *120)
print("파일 저장 완료")
print("=" *120)

파일 저장 완료
